In [ ]:
# load the trained generator-model
fake_data_model= load_model('/home/hashemi/Fei Shao/WGAN_2.5_100epoch.h5')
batch_size = 1700 # Adjust parameters to generate any number of fake data 
noise = np.random.normal(loc=0, scale=1, size=(batch_size, 8)) # feed noise
generated_data = fake_data_model.predict(noise)

# save generated_data(fake data) before data cleaning into Excel
data = pd.DataFrame(generated_data)
data.to_excel('/home/hashemi/Fei Shao/fake_2.5.xlsx', index=False)

# read fake data
faketrain_data = pd.read_excel('/home/hashemi/Fei Shao/fake_2.5.xlsx', dtype=np.float32)
faketrain_data

# data cleaning (WGAN trained 3 generator model for particle 1.5mm, 2.5mm, 3.5mm,
#so there needs to do 3 data cleaning for each size particle.)
from decimal import Decimal
# restore magnitude order
faketrain_data.loc[:,0] = 2.5 # if gennrator model is for all size, like flowrate in SME,here need to be some changed
faketrain_data.iloc[:,1] = (faketrain_data.iloc[:,1]*100).astype(int)/100
faketrain_data.iloc[:,2] = (faketrain_data.iloc[:,2]*100).astype(int)/100
faketrain_data.iloc[:,3] = round(faketrain_data.iloc[:,3]/100000, 3)
faketrain_data.iloc[:,4] = (faketrain_data.iloc[:,4]*10).astype(int)
faketrain_data.iloc[:,5] = (faketrain_data.iloc[:,5]/100).astype(int)
faketrain_data.iloc[:,6] = round(faketrain_data.iloc[:,6]/10000, 2)
faketrain_data.iloc[:,7] = round(faketrain_data.iloc[:,7]/10000, 2) 
# delete all rows with negative values
faketrain_data = faketrain_data[(faketrain_data > 0).all(axis=1)] 
# delete all rows where the friction coefficient is greater than 1 (Remove parameters that do not conform to physics)
faketrain_data = faketrain_data.drop(faketrain_data[faketrain_data.iloc[:,6]>1].index)  

# save fakedata after data cleaning
faketrain_data.reset_index(drop=True, inplace=True)
faketrain_data
faketrain_data.to_excel('/home/hashemi/Fei Shao/fake_2.5clean_.xlsx', index=False)

# combine the fake data together
df1 = pd.read_excel('/home/hashemi/Fei Shao/fake_1.5clean_.xlsx')
df2 = pd.read_excel('/home/hashemi/Fei Shao/fake_2.5clean_.xlsx')
df3 = pd.read_excel('/home/hashemi/Fei Shao/fake_4.5clean_.xlsx')
frames = [df1, df2, df3]
df = pd.concat(frames, ignore_index=True)

# combine the fake data set with original data set
df0 = pd.read_excel('/home/hashemi/Fei Shao/total.xlsx')
frames = [df0, df]
dataframe = pd.concat(frames)
dataframe.to_excel('/home/hashemi/Fei Shao/total_fake.xlsx', index=False)